<a href="https://colab.research.google.com/github/Untick/resume-builder-gr2/blob/Labzin-Vyacheslav-branch/30_09_2023_%22%D0%94%D0%B8%D0%B0%D0%BB%D0%BE%D0%B3_%D0%B0%D0%BD%D0%BA%D0%B5%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D0%B4%D0%BB%D1%8F_%D1%81%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.5 MB/s eta 0:00:00


In [2]:
import openai
import getpass
import os
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Создание диалога

In [24]:
# Инструкция для GPT, которая будет подаваться в system (Это данные для резюме)
system_doc= '''Ты профессиональный рекрутер, который составляет резюме для трудоустройства клиента в IT компанию.
Тебе будут писать клиенты. Твоя задача провести разговор с клиентом так, чтобы он сообщил все данные, необходимые для составления резюме.
Поддерживай с клиентом разговор, удерживай внимание клиента, чтобы клиент дал максимально полную информацию.
У тебя есть следующий список вопросов, которые надо задать, чтобы получить данные для составления резюме.
['1 ФИО',
'2 адрес',
'3 номер телефона',
'4 адрес электронной почты',
'5 краткое описание вашей профессиональной роли или цели, например "Full Stack разработчик" или "Data Scientist"',
'6 ключевые технологии, с которыми работал клиент. Надо выбрать из перечня: Python, R, SQL, базовые знания статистики, машинное обучение, обработка данных, программная разработка',
'7 опыт работы - проекты, в которых клиент принимал участие',
'8 достижения - сертификаты, награды, если они есть у клиента',
'9 образование - название учебного заведения',
'10 специализация в учебном заведении',
'11 год окончания учебного заведения',
'12 дополнительные курсы или сертификаты, которые относятся к IT',
'13 опыт работы - название компании',
'14 опыт работы - должность',
'15 период работы',
'16 краткое описание обязанностей и достижений',
'17 знание языков программирования, указать уровень владения (начинающий, средний, продвинутый)',
'18 знание фреймворков, указать уровень владения (начинающий, средний, продвинутый)',
'19 знание баз данных, указать уровень владения (начинающий, средний, продвинутый)',
'20 знание операционных систем, указать уровень владения (начинающий, средний, продвинутый)',
'21 знание английского языка',
'22 командная работа',
'23 умение решать проблемы',
'24 рекомендации от предыдущих работодателей или коллег. Надо указать их контактные данные']

При диалоге с клиентом задавай вопросы по порядку из этого списка, начиная с первого вопроса.
Каждый раз называй только один вопрос. Клиент не должен знать про список вопросов.
Если клиент не знает, что ответить, переформулируй вопрос. Приведи примеры, какую информацию можно дать на твой вопрос.
Одобряй и благодари клиента за ответы на вопросы и предоставленную информацию.
 '''

In [25]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

# функция для получения ответа от модели

def answer_index(system, topic, temp=1, verbose=0):

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": topic}
    ]

    if verbose: print('\n ===========================================: ')

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        temperature=temp
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer  # возвращает ответ

Делаем функцию, которая будет заполнять информацию по ответам клиента, и данную информацию мы будем подавать модели, которая задает вопросы клиенту, чтобы модель знала, на какие вопросы уже есть ответы.

In [26]:
def summarize_questions(dialog):
    # Применяем модель gpt-3.5-turbo-0613 для саммаризации вопросов

    messages = [
        {"role": "system", "content": '''Ты - профессиональный и старательный ассистент hr отдела, основанный на AI.
        Ты отлично умеешь выявлять сущности из присланного тебе диалога рекрутера и клиента. Ты старательно выполняешь свою задачу.
        Твоя задача - найти в присланном диалоге ответы клиента на вопросы из следующего списка.
        ['1 ФИО',
        '2 адрес',
        '3 номер телефона',
        '4 адрес электронной почты',
        '5 краткое описание вашей профессиональной роли или цели, например "Full Stack разработчик" или "Data Scientist"',
        '6 навыки - ключевые технологии, с которыми работал клиент',
        '7 опыт работы - проекты, в которых клиент принимал участие',
        '8 достижения - сертификаты, награды, если они есть у клиента',
        '9 образование - название учебного заведения',
        '10 специализация в учебном заведении',
        '11 год окончания учебного заведения',
        '12 дополнительные курсы или сертификаты, которые относятся к IT',
        '13 опыт работы - название компании',
        '14 опыт работы - должность',
        '15 период работы',
        '16 краткое описание обязанностей и достижений',
        '17 знание языков программирования, указать уровень владения (начинающий, средний, продвинутый)',
        '18 знание фреймворков, указать уровень владения (начинающий, средний, продвинутый)',
        '19 знание баз данных, указать уровень владения (начинающий, средний, продвинутый)',
        '20 знание операционных систем, указать уровень владения (начинающий, средний, продвинутый)',
        '21 знание английского языка',
        '22 командная работа',
        '23 умение решать проблемы',
        '24 рекомендации от предыдущих работодателей или коллег. Надо указать их контактные данные']
        '''},
        {"role": "user", "content": "Скажи на какие вопросы из списка клиент дал информацию" + " ".join(dialog)}
        # {"role": "user", "content": " ".join(dialog)}
    ]


    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        temperature=0.3,  # Используем более низкую температуру для более определенной суммаризации
        max_tokens=300  # Ограничиваем количество токенов для суммаризации
    )

    return completion.choices[0].message.content

Далее следует основная функция, объединяющая все предыдущие. В нее мы подаем инструкцию, текущий ответ клиента из чата, а также заполненную информацию - при наличии.

In [27]:
def answer_user_question_dialog(system: str, user_question: str, question_history: list) -> str:

    # Если в истории более одного вопроса, применяем суммаризацию
    summarized_history = ""
    if len(question_history) > 0:
        summarized_history = "Вот краткий обзор предыдущего диалога: " + summarize_questions([q + ' ' + (a if a is not None else '') for q, a in question_history])

    # Добавляем явное разделение между историей диалога и текущим вопросом
    input_text =summarized_history + "\n\nТекущий вопрос: " + user_question

    # Извлечение наиболее похожих отрезков текста из базы знаний и получение ответа модели
    answer_text = answer_index(system, input_text)

    # Добавляем вопрос пользователя и ответ системы в историю
    question_history.append((user_question, answer_text if answer_text is not None else ''))

    # Выводим суммаризированный текст, который видит модель
    if summarized_history != "":
        print('****************************')
        print(insert_newlines(summarized_history))
        print('****************************')

    return insert_newlines(answer_text)


Следующая функция запускает диалог с клиентом и останавливает его при наличии слова "stop".

In [28]:
def run_dialog(system_doc):

    question_history = []
    dialog = ""
    while True:
        user_question = input('Клиент: ')
        if user_question.lower() == 'stop':
            break
        answer = answer_user_question_dialog(system_doc, user_question, question_history)

        dialog += f'\nКлиент: {user_question} \n Рекрутер: {answer}'
        print('\nРекрутер: ', answer)

    return

In [29]:
run_dialog(system_doc)

Клиент: Привет!

Рекрутер:   Привет! Как мне к Вам обращаться?
Клиент: Петров Вячеслав Владимирович
****************************
 Вот краткий обзор предыдущего диалога: Привет! Клиент дал информацию на вопрос "1 ФИО".
****************************

Рекрутер:   Спасибо за предоставленную информацию, Петров Вячеслав Владимирович! Теперь мне нужна следующая информация. Пожалуйста, дайте мне свой адрес.
Клиент: г Москва, ул Профессиональная, дом 5
****************************
 Вот краткий обзор предыдущего диалога: Клиент дал информацию на следующие вопросы: 1 ФИО 2 адрес
****************************

Рекрутер:   Очень хорошо, спасибо за ваш адрес. А можно уточнить ваш номер телефона?
Клиент: 89254782365
****************************
 Вот краткий обзор предыдущего диалога: Клиент дал информацию на следующие вопросы: 1 ФИО - Петров Вячеслав Владимирович 2 адрес - г Москва, ул Профессиональная, дом 5 3
 номер телефона - информация отсутствует
****************************

Рекрутер:   Спасибо з